In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
#connect to files 'applications.csv' and 'industries.csv'
url='https://drive.google.com/file/d/1XaRWDDRiw2sSjEorwZ5Pi-ebpJqr1Mk4/view?usp=sharing'
url2='https://drive.google.com/file/d/1dC1GhCGP8wx66r6C-kmeSCT0RlTH3s37/view?usp=sharing'
df_app = pd.read_csv('https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url.split('/')[-2]))
df_ind = pd.read_csv('https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url2.split('/')[-2]))
df_app.info()
df_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13315 entries, 0 to 13314
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       13315 non-null  object 
 1   Amount           13272 non-null  float64
 2   Age              13315 non-null  int64  
 3   Gender           13315 non-null  object 
 4   Industry         13315 non-null  object 
 5   Marital status   13315 non-null  object 
 6   External Rating  13243 non-null  float64
 7   Education level  13282 non-null  object 
 8   Location         11540 non-null  object 
 9   applicant_id     13315 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Industry  37 non-null     object
 1   Score     37 non-null     int64 
dtypes: int64(1), objec

Task 1: Прибери дублікати applicant_id

In [5]:
# Прибери дублікати applicant_id
df_app_cleared = df_app.drop_duplicates(subset=['applicant_id'])
df_app_cleared.info()
# Скільки унікальних заявок залишилось?
df_app_cleared.applicant_id.nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 13278 entries, 0 to 13314
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       13278 non-null  object 
 1   Amount           13245 non-null  float64
 2   Age              13278 non-null  int64  
 3   Gender           13278 non-null  object 
 4   Industry         13278 non-null  object 
 5   Marital status   13278 non-null  object 
 6   External Rating  13206 non-null  float64
 7   Education level  13245 non-null  object 
 8   Location         11506 non-null  object 
 9   applicant_id     13278 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.1+ MB


13278

Task 2: В полі 'External Rating' заповни відсутні значення нулями.

In [6]:
# В полі 'External Rating' заповни відсутні значення нулями.

#код-> df_app_cleared['External Rating'] = df_app_cleared['External Rating'].fillna(0)
#Маємо попередження: A value is trying to be set on a copy of a slice from a DataFrame.
#Try using .loc[row_indexer,col_indexer] = value instead. 
# Тому робимо так:
df_app_cleared.loc[:, 'External Rating'] = df_app_cleared.loc[:, 'External Rating'].fillna(0)

# Перевірка, що відсутніх значень більше немає
df_app_cleared['External Rating'].isna().sum()

np.int64(0)

In [7]:
# Перевірка, скільки є незаповнених значень в полі 'Education level'
df_app_cleared['Education level'].isna().sum()

np.int64(33)

Task 3: В полі 'Education level' заповни відсутні значення текстом “Середня”

In [8]:
# В полі 'Education level' заповни відсутні значення текстом “Середня”
df_app_cleared.loc[:, 'Education level'] = df_app_cleared.loc[:, 'Education level'].fillna("Середня")

# Перевірка, що відсутніх значень більше немає
df_app_cleared['Education level'].isna().sum()

np.int64(0)

Task 4: Додай до цього DataFrame дані з файлу industries.csv, а саме, рейтинги індустрій.

In [9]:
# Task 4: Додай до цього DataFrame дані з файлу industries.csv, а саме, рейтинги індустрій.
df = pd.merge(df_app_cleared, df_ind, on='Industry', how='left')
df.head(3)

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f,20
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10


**Task 5: Розрахуй рейтинг заявки за наступними умовами:** 

- Рейтинг має бути числом від 0 до 100
- Рейтинг - сума оцінок заявки по 6 критеріям.
- Рейтинг дорівнює нулю, якщо відсутнє значення 'Amount' або якщо 'External Rating' дорівнює нулю.
___
З чого складається рейтинг:
1. Якщо вік заявника між 35 та 55, до рейтингу додається 20 балів
2. Якщо заявка була подана не у вихідні, до рейтингу додається 20 балів
3. Якщо заявник одружений, до рейтингу додається 20 балів
4. Якщо заявник знаходиться в Києві чи області, до рейтингу додається 10 балів

Значення 'Score' з таблиці industries.csv також додається до заявки (і складає від 0 до 20 балів)
1. Якщо 'External Rating' більше чи дорівнює 7, до рейтингу додається 20 балів
2. Якщо 'External Rating' менше чи дорівнює 2, з рейтингу віднімається 20 балів
___
*Підказка: Ти можеш множити булеві масиви на цілі числа і отримувати масиви з цілих чисел. Наприклад, на своїй таблиці “df” виконай команду нижче, і ти отримаєш масив з цілими числами, де значення кожного елементу буде дорівнювати або 0, або 20. (df['Age'] >= 35) * 20 І далі, ці масиви ти можеш додавати між собою, та зберегти результат як новий стовпчик в DataFrame.*  

In [ ]:
# Створити нове поле 'Age_score', яке буде містити 20 (балів), 
# якщо вік (поле 'Age') знаходиться в діапазоні від 35 до 55 років, і 0 в іншому випадку.
df['Age_score'] = ((df['Age']>35) & (df['Age']<55)) * 20
# Приводимо поле 'Applied at' до формату datetime
df['Applied at'] = pd.to_datetime(df['Applied at'], format = 'mixed')
# Створити нове поле 'Not_weekend', Якщо заявка була подана не у вихідні, до рейтингу додається 20 балів
df['Not_weekend'] = ((df['Applied at'].dt.dayofweek < 5)) * 20
# створити поле 'Married_score' - Якщо заявника одружений, до рейтингу додається 20 балів
df['Married_score'] = (df['Marital status'] == 'Married') * 20
# створити поле "Kyiv_loc" - Якщо заявник знаходиться в Києві чи області, до рейтингу додається 10 балів
df['Kyiv_loc'] = df['Location'].str.contains('Київ', na=False) * 10
#створити полн 'Ext_Rate_score' і обчислити його
df['Ext_Rate_score'] = 0
df.loc[df['External Rating'] >= 7, 'Ext_Rate_score'] = 20
df.loc[df['External Rating'] <= 2, 'Ext_Rate_score'] = -20

# Cтворити нове поле 'Rate', яке буде містити значення з поля 'Score' плюс суму всіх інших полів, що формують рейтинг
df['Rate'] = df['Score'] + df['Age_score'] + df['Not_weekend'] + df['Married_score'] + df['Kyiv_loc'] + df['Ext_Rate_score']

# Check - рейтинг дорівнює нулю, якщо відсутнє значення 'Amount' або якщо 'External Rating' дорівнює нулю
df.loc[ df['Amount'].isna() | (df['External Rating'] == 0), 'Rate'] = 0
# Обмежити масив діапазоном від 0 до 100, все що менше 0 = 0, що більше 100 = 100
df['Rate'] = df['Rate'].clip(0,100)
#df.loc[ df['Rate'] >= 100]
df.head(5)

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score,Age_score,Not_weekend,Married_score,Kyiv_loc,Ext_Rate_score,Rate
0,2022-11-30 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0,0,20,0,0,20,40
1,2022-11-30 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f,20,20,20,0,0,0,0
2,2022-11-30 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10,0,20,0,10,0,40
3,2022-11-30 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1,15,0,20,0,0,-20,0
4,2022-11-30 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15,0,20,0,0,0,35


Task 4 - В результуючій таблиці залиш лише заявки з рейтингом більше нуля, ці заявки вважатимуться прийнятими.

In [15]:
# Filter rows with rate > 0. Create new df
accepted_df = df.loc[df['Rate'] > 0].copy()
accepted_df.head(4) 


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score,Age_score,Not_weekend,Married_score,Kyiv_loc,Ext_Rate_score,Rate
0,2022-11-30 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0,0,20,0,0,20,40
2,2022-11-30 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10,0,20,0,10,0,40
4,2022-11-30 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15,0,20,0,0,0,35
5,2022-11-30 10:27:38,16500.0,31,Чоловік,E-commerce,Single,8.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,f720bf9c5c4c3e10a8568c1699847696,15,0,20,0,10,20,65


Task 5 - Дані з результуючої таблиці згрупуй за тижнем подачі заявки, та виведи середній рейтинг прийнятих заявок у кожен тиждень.

In [14]:
# Group by week of applying
weekly_df = accepted_df.groupby(pd.Grouper(key="Applied at", freq="W"))["Rate"].mean().round(2).reset_index()
weekly_df


,Applied at,Rate
0,2022-12-04,49.70
1,2022-12-11,48.84
2,2022-12-18,48.93
3,2022-12-25,46.14
4,2023-01-01,50.06
5,2023-01-08,50.47
6,2023-01-15,51.88
